In [1]:
!pip install -q langchain_community
!pip install -q pypdf
!pip install -q -U langchain-text-splitters
!pip install -q -U langchain
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U chromadb
!pip install -q -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM,
import torch
from transformers import BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import LlamafileEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Meta-Llama-3-8B"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
from langchain_community.document_loaders import PyPDFLoader

pdf = PyPDFLoader("/content/drive/MyDrive/llm/planet.pdf")
pages = pdf.load_and_split()


In [7]:
template = """
<s>[INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context.
If the answer is not in the context, reply "I dont know".
Context: {context}
<</SYS>>
Query: {query} [/INST]
"""

prompt = PromptTemplate.from_template(template)
fprompt = prompt.format(context="My give name is venkata ratnam, my friends call me Venkat", query="what is my name ?")

fprompt

'\n<s>[INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context.\nIf the answer is not in the context, reply "I dont know".\nContext: My give name is venkata ratnam, my friends call me Venkat\n<</SYS>>\nQuery: what is my name ? [/INST]\n'

In [15]:
messages = [
    {"role": "system", "content": "My give name is venkata ratnam, my friends call me Venkat"},
    {"role": "user", "content": "what is my name ?"},
]


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

generate_ids = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
).to(device)

generated_ids = generate_ids[0][input_ids.shape[-1]:]

ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print (ans)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

My give name is venkata ratnam, my friends call me Venkatuser

what is my name ?assistant

Your name is Venkata Ratnam, but your friends call you Venkat.


In [16]:
messages = [
    {"role": "system", "content": "Answer the question only based on details in the context string below but dont mention about context. If the answer is not in the context, reply - I dont know."},
    {"role": "user", "content":"My give name is venkata ratnam, my friends call me Venkat"},
    {"role": "user", "content": "what is the size of pacific ocean?"},
]


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

generate_ids = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
).to(device)

generated_ids = generate_ids[0][input_ids.shape[-1]:]

ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print (ans)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Answer the question only based on details in the context string below but dont mention about context. If the answer is not in the context, reply - I dont know.user

My give name is venkata ratnam, my friends call me Venkatuser

what is the size of pacific ocean?assistant

I dont know.


In [17]:
# Define the Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(pages)
tokenizer.pad_token = tokenizer.eos_token
persist_directory = './docs/chroma/'
default_ef = embedding_functions.DefaultEmbeddingFunction()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
vectordb = Chroma.from_documents(
    documents=text_splitter.split_documents(pages),
    embedding=embeddings,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

8


In [36]:
question = "what is the technology level of Celestrians ?"

# Similarity search with k = 5
docs = vectordb.similarity_search(question,k=10)

context = ""

for d in docs:
  context += d.page_content

messages = [
    {"role": "system", "content": "Answer the question only based on details in the context string below but dont mention about context. If the answer is not in the context, reply - I dont know."},
    {"role": "user", "content": context},
    {"role": "user", "content": question},
]


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

generate_ids = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
).to(device)

ans = tokenizer.batch_decode(generated_ids.unsqueeze(0))[0]

print ("Answer: \n", ans)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: 
 The technology level of the Celestrians is comparable to an advanced Type II civilization on the Kardashev scale.<|eot_id|>


In [ ]:
question = "Am i subscribed to MTV channel ?"

context = """
Account name: John Doe
Subsciptions: Premium,
Premium subscription channels: MTV, CNN, Canal+, ESPN, Star sports
"""

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt").to(device)

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-1:])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


]
Answer: Yes, you are subscribed to MTV channel.


In [53]:
question = "Am i subscribed to BBC ?"

context = """
Context :

Account name: John Doe
Subsciptions: Premium product
User is subscribed to channels MTV, CNN, Canal+, ESPN, Star sports only and other channels are not suscribed.
"""

messages = [
    # {"role": "system", "content": "Answer the question only based on details provided in system context. If the answer is not in the context, reply - I dont know."},
    {"role": "user", "content": context},
    {"role": "user", "content": question},
]


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
).to(device)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


According to John Doe's subscription information, the answer is: No, John Doe is not subscribed to BBC.


In [54]:
question = "I took box C home, what are colors of balls i have ?"

context = """
There are three boxes A, B and C. Box A contains red and green color balls.
Box B contains red and yellow balls.
Box C contains green and blue balls.
"""


messages = [
    # {"role": "system", "content": "Answer the question only based on details provided in system context. If the answer is not in the context, reply - I dont know."},
    {"role": "user", "content": context},
    {"role": "user", "content": question},
]


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
).to(device)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Congratulations on choosing box C!

Since box C contains green and blue balls, you have a chance to find either green or blue balls when you open it.

There's no way to know for sure without opening the box, but that's the mystery and excitement of it!
